In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import math
import h5py
import sys
sys.path.append('/eos/home-e/epuljak/private/epuljak')
sys.path.append('../')
sys.path.append('../../')
import setGPU

# import qibo
# qibo.set_backend("tensorflow")

# import scripts.qkmeans as qkm
#import scripts.minimization as m
#import scripts.oracle as o
#import scripts.grover as g
#import scripts.qkmedians as qkmed
import scripts.distance_calc as distc
import scripts.util as ut
#import utils as u
#import plots as p

In [ ]:
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.visualization.utils import _bloch_multivector_data
from qiskit import QuantumCircuit, execute, Aer, transpile
#from qiskit.tools.jupyter import *
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, Aer, IBMQ, execute
#from qiskit.tools.monitor import job_monitor

In [ ]:
# qibo.set_device("/GPU:0")

In [ ]:
from math import remainder
a = [3.4, 2.5, 1.9, 0.2, 10.1]
a=ut.normalize(a)
print(a)
for i in a:
    print(remainder(i, math.pi/2))

In [ ]:
a = np.array([1,5,6,1,7,4,6,9], dtype=np.float32)
b = np.array([2,3,7,4,1,2,2,2], dtype=np.float32)
num_features = 8

In [ ]:
distance_TI, qc = distc.DistCalc_DI(a, b)
distance_TI

In [ ]:
distances = np.array([4.5, 6.2, 7.0, 9.0, 4.2, 5.9, 10.2, 1.6])

In [ ]:
def amplitude_encoding(xi):
    xi = ut.normalize(xi)
    xi = distc.pad_input(xi)
    print(xi)
    n_qubits = math.log2(len(xi))
    return xi, n_qubits

In [ ]:
ampl, n_qubits = amplitude_encoding(distances)

In [ ]:
n_qubits

In [ ]:
a = QuantumRegister(n_qubits, "a")
c_a = ClassicalRegister(n_qubits, "cr")

# Creating Quantum Circuit called "qc" involving your Quantum Register "qr"
# and your Classical Register "cr"
qc = QuantumCircuit(a, c_a, name="k_means")
qc.initialize(ampl, range(int(n_qubits)))
#qc.measure(a, c_a)
state_vec = Statevector.from_instruction(qc)
state_vec.to_dict()

In [ ]:
plot_bloch_multivector(state_vec)

In [ ]:
from qiskit.visualization.utils import _bloch_multivector_data

bloch_data = (_bloch_multivector_data(state_vec))
print(bloch_data)
qubitvector_0=bloch_data[0] # qubit 0
x_0=qubitvector_0[0]
y_0=qubitvector_0[1]
z_0=qubitvector_0[2]

qubitvector_1=bloch_data[1] # qubit 1
x_1=qubitvector_1[0]
y_1=qubitvector_1[1]
z_1=qubitvector_1[2]

qubitvector_2=bloch_data[2] # qubit 2
x_2=qubitvector_2[0]
y_2=qubitvector_2[1]
z_2=qubitvector_2[2]

In [ ]:
#tan𝜑=𝑦/𝑥
#cos𝜃=𝑧
phi_0 = math.atan(y_0/x_0)
theta_0 = math.acos(z_0)

phi_1 = math.atan(y_1/x_1)
theta_1 = math.acos(z_1)

phi_2 = math.atan(y_2/x_2)
theta_2 = math.acos(z_2)

In [ ]:
phi_0, theta_0

In [ ]:
a = QuantumRegister(6, "a")
c_a = ClassicalRegister(6, "cr")

# Creating Quantum Circuit called "qc" involving your Quantum Register "qr"
# and your Classical Register "cr"
qc = QuantumCircuit(a, c_a, name="k_means")
qc.h(a)
#qc.measure(a, c_a)
state_vec = Statevector.from_instruction(qc)
state_vec.to_dict()

In [ ]:
int(10.4)

## Convert distances to binary fixed-point precision

In [ ]:
import scripts.minimization as m
import h5py
import math
import scripts.qkmedians as qkmed
import qibo
qibo.set_backend("tensorflow")

In [ ]:
latent_dim='24'
run = '01032022_1'
test_size=100
save_dir='/eos/user/e/epuljak/private/epuljak/PhD/TN/QIBO/search_algorithms/notebooks/results_qmedians/corrected_cuts'

In [ ]:
# read QCD predicted data (test - SIDE)
read_dir =f'/eos/user/e/epuljak/private/epuljak/public/diJet/{latent_dim}/'
file_name = f'{read_dir}latentrep_QCD_sig_testclustering.h5'
with h5py.File(file_name, 'r') as file:
    data = file['latent_space']
    l1 = data[:,0,:]
    l2 = data[:,1,:]
    #r_index = np.random.choice(list(range(l1.shape[0])), size=int(test_size/2))
    data_test_qcd = np.vstack([l1[:test_size], l2[:test_size]])

In [ ]:
k = 4 # number of clusters
centroids = qkmed.initialize_centroids(data_test_qcd, k)   # Intialize centroids
tolerance=1e-8

In [ ]:
maxs=[]
for i in range(data_test_qcd.shape[1]):
    maxs.append(max(data_test_qcd[:, i]))
max(maxs)

In [ ]:
mins=[]
for i in range(data_test_qcd.shape[1]):
    mins.append(min(data_test_qcd[:, i]))
min(mins)

In [ ]:
math.sqrt(2**3)

In [ ]:
cluster_label, distances = qkmed.find_nearest_neighbour_DI(data_test_qcd,centroids)

In [ ]:
maxs=[]
for i in range(distances.shape[1]):
    maxs.append(max(distances[:, i]))
max(maxs)

In [ ]:
mins=[]
for i in range(distances.shape[1]):
    mins.append(min(distances[:, i]))
min(mins)

In [ ]:
import scripts.classic_functions as cf
cluster_label_c, distances_c = cf.find_nearest_neighbour_classic(data_train,centroids)
#distances_c

In [ ]:
maxs=[]
for i in range(data_train.shape[0]):
    maxs.append(max(distances[i, :]))
print(max(maxs))

In [ ]:
dist_norm = []; norm=[]
for dist in distances[:]:
    d = ut.normalize(dist)
    norm.append(np.linalg.norm(dist))
    dist_norm.append(d)
    #dist_norm.append([math.remainder(a, math.pi/2.) for a in d])

In [ ]:
dist = w

In [ ]:
# for j in range(distances.shape[0]):
    
#     qc = QuantumCircuit(n_qubits, n_qubits, name='neg_rot_circ')
#     # rotation for point
#     for i in range(n_qubits):
#         qc.ry(dist_norm[j][i], i)
#     #state_vec = Statevector.from_instruction(qc)
#     #plot_bloch_multivector(state_vec)
    
#     for i in range(n_qubits):
#         qc.measure(i, i)
    
#     simulator = Aer.get_backend('qasm_simulator')
#     circ = transpile(qc, simulator)

#     # Run and get counts
#     result = simulator.run(qc, shots=10000).result()
#     counts = result.get_counts(qc)
#     print(counts)
#     count_1 = (counts['00'] + counts['01'])/10000
#     count_2 = (counts['00'] + counts['10'])/10000

#     new_cluster_label.append(np.argmin([count_1, count_2]))

In [ ]:
# simulator = Aer.get_backend('qasm_simulator')
# circ = transpile(qc, simulator)

# # Run and get counts
# result = simulator.run(qc, shots=1024).result()
# counts = result.get_counts(qc)
# count_1 = (counts['00'] + counts['01'])/1024
# count_2 = (counts['00'] + counts['10'])/1024

# count_1, count_2

In [ ]:
# to fixed-point representation

# max distance = 2*sqrt(N) where N=lat_dim because max value = sqrt((1+1)^2*N)
max_dist = 2*math.sqrt(lat_dim)
print(max_dist+1)
n_bits = int((math.log(max_dist+1)/math.log(2)) + 1) # can varry the performance because of this!
print(n_bits)
step = max_dist+1
step

In [ ]:
precision=2**n_bits

In [ ]:
n_qubits=[]
dist_fix = []; dist_fix_float=[]
for i in range(distances.shape[0]):
    max_dist = math.ceil(max(distances[i,:]))
    n_bits = int((math.log(max_dist)/math.log(2)) + 1)+1
    step = max_dist/(2**n_bits)
    n_qubits.append(n_bits)
    dist_fix.append([int((distances[i,0]/step)),\
                     int((distances[i,1]/step)),\
                     int((distances[i,2]/step))])
    dist_fix_float.append(distances[i,:]/max_dist)
dist_fix[:10], dist_fix_float[:10]

In [ ]:
n_qubits[:10]

## BASIS ENCODING
#### Define reference state $|a\rangle$ and superposition of all others $|b\rangle$

In [ ]:
def basis_encoding(inputs, n_qubits):
    amplitudes = np.zeros(2**n_qubits)
    for i in inputs:
        amplitudes[i] = 1./math.sqrt(len(inputs))
    return amplitudes

In [ ]:
n_qubits[0]

In [ ]:
# a - threshold
index_rand = np.random.choice(list(range(k))) # random index of reference state
print(index_rand)
threshold = dist_fix[0][index_rand]

# qc = QuantumCircuit(n_qubits[0], n_qubits[0])
# ampl = basis_encoding([threshold], n_qubits[0])
# qc.initialize(ampl)
# state_vec_a = Statevector.from_instruction(qc)
# qc.draw()

In [ ]:
threshold

In [ ]:
# b - superposition
qc = QuantumCircuit(n_qubits[0], n_qubits[0])
ampl = basis_encoding(dist_fix[0][:], n_qubits[0])
qc.initialize(ampl)
state_vec_b = Statevector.from_instruction(qc)
#qc.measure(range(n_qubits[0]), range(n_qubits[0]))
qc.draw()

In [ ]:
state_vec.to_dict()

## QRAM DATA ENCODING 
#### Extract values for reference state $|a\rangle$ and superposition of all others $|b\rangle$

In [ ]:
from qiskit.quantum_info import Statevector
def qram_encoding(inputs):
    inputs = distc.pad_input(inputs)
    N = len(inputs)
    n_qubits = int(math.log2(N))
    
    # statevector 1
    address = QuantumRegister(n_qubits, "address")
    qc_1 = QuantumCircuit(address, name='address_circ')
    qc_1.h(address)
    state_vec_a = Statevector.from_instruction(qc_1)
    print(state_vec_a.to_dict())
    
    # statevector 2
    data_values = QuantumRegister(n_qubits, "values") # with Ampl. Encoding
    data_values_ampl, _ = amplitude_encoding(inputs)
    qc_2 = QuantumCircuit(data_values, name='inputs_circ')
    qc_2.initialize(data_values_ampl, data_values)
    state_vec_v = Statevector.from_instruction(qc_2)
    print(state_vec_v.to_dict())

    state_vec_comb = state_vec_a.tensor(state_vec_v)
    print(state_vec_comb.to_dict())
    data = state_vec_comb.data
    new_data=[]
    for i, d in enumerate(data):
        if i == 0 or i % (2**n_qubits + 1)==0:
            new_data.append(d)
        else:
            new_data.append(0 + 0j)
    new_data = ut.normalize(new_data) # imaginary part is discarded - watchout for later

    state_vec_init = Statevector(data=new_data, dims=state_vec_comb.dims())
    print(state_vec_init.to_dict())
    # combined circuit
    #qr_1 = QuantumRegister(n_qubits, "a")
    #qr_2 = QuantumRegister(n_qubits, "v")
    qc_comb = QuantumCircuit(n_qubits*2, name='qram_encoding')
    qc_comb.initialize(state_vec_init)

    state_vec = Statevector.from_instruction(qc_comb)
    plot_bloch_multivector(state_vec)
    print(state_vec.to_dict()) # slight change in values from state_vec_init - maybe it matters check!
    return qc_comb, state_vec, state_vec_init

In [ ]:
inputs = ut.normalize(inputs)
np.round_(inputs, decimals=4)

In [ ]:
inputs = np.array([1.3,2.2,5.3,0.4])
k = len(inputs)
n_qubits = int(math.log2(k))

In [ ]:
n_qubits

In [ ]:
qc, _, state_vector_b = qram_encoding(inputs)

In [ ]:
plot_bloch_multivector(state_vector_b)

In [ ]:
def get_per_qubit_params(statevector, n_qubits):
    bloch_data = (_bloch_multivector_data(statevector))
    print(bloch_data)
    
    phi=[]; theta=[]
    x=[]; y=[]; z=[]
    for n in range(n_qubits):
        qubit_vec = bloch_data[n]
        x.append((qubit_vec[0]))
        y.append((qubit_vec[1]))
        z.append((qubit_vec[2]))
        if qubit_vec[0] == 0.0: phi.append(0.)
        else: phi.append(math.atan(qubit_vec[1]/qubit_vec[0]))
        theta.append(math.acos(qubit_vec[2]))
    return x, y, z, phi, theta     # because you can use RVGate to rotate using x, y, z


In [ ]:
# get statevector bit by bit for \a>
index_rand = np.random.choice(list(range(k))) # random index of reference state
print(index_rand)
#threshold = inputs[index_rand]

thresh_bin = np.binary_repr(int(index_rand), width=int(n_qubits))
reference_state_string = str(thresh_bin)*2
reference_state_ampl = state_vector_b.to_dict()[reference_state_string]
print(reference_state_string)

amplitudes = np.zeros(2**(n_qubits*2))
print(amplitudes)
amplitudes[int(reference_state_string,2)] = reference_state_ampl
print(amplitudes)
#amplitudes = ut.normalize(amplitudes) # --- then your statevector is 100% in this state, needs to be

state_vec_init_a = Statevector(data=amplitudes)
print(state_vec_init_a.to_dict())

#qc_a = QuantumCircuit(n_qubits*2)
#qc_a.initialize(state_vec_init_a)

x_a, y_a, z_a, phi_a, theta_a = get_per_qubit_params(state_vec_init_a, n_qubits*2)
x_a, y_a, z_a, phi_a, theta_a

In [ ]:
plot_bloch_multivector(state_vec_init_a)

In [ ]:
# get statevector bit by bit for \b>

x_b, y_b, z_b, phi_b, theta_b = get_per_qubit_params(state_vector_b, n_qubits*2)
x_b, y_b, z_b, phi_b, theta_b

In [ ]:
# for basis encoding

# a
#_,_,_, phi_a, theta_a = get_per_qubit_params(state_vec_a, n_qubits[0])

# b
_,_,_, phi_b, theta_b = get_per_qubit_params(state_vec_b, n_qubits[0])

## Define Quantum Bit Comparator Circuit (QBSC)

In [ ]:
from qiskit.circuit.library.generalized_gates.rv import RVGate

In [ ]:
n_qubits=2

In [ ]:
n_qubits_qbsc = n_qubits[0]*2 + (3*(n_qubits[0]-1) + 2)
n_qubits_qbsc, n_qubits[0]

In [ ]:
n_qubits_arr = n_qubits
n_qubits = n_qubits_arr[0]

In [ ]:
n_qubits

In [ ]:
bin_thresh = np.binary_repr(int(threshold), width=int(n_qubits))
bin_thresh = list(bin_thresh)
#bin_thresh = list(new_threshold) # from below
bin_thresh

In [ ]:
#cr_a=ClassicalRegister(2,name='a')
#cr_b=ClassicalRegister(2, name='b')

qc_qbsc = QuantumCircuit(n_qubits_qbsc+2, n_qubits, name='QBSC') # +1 = ancilla for grover |+> + ancilla for oracle
i=0
j_a=0; assign_a=False
j_b=0; assign_b=True
while i < n_qubits_qbsc:
    if not assign_a:
        print(i)
        #qc_qbsc.rv(x_a[j_a],y_a[j_a], z_a[j_a], qubit=i)
        if bin_thresh[j_a]=='1':
            qc_qbsc.x(i)
        #qc_qbsc.(theta_a[j_a],phi_a[j_a], qubit=i)
        j_a+=1
        assign_a = True
        assign_b = False
        i+=1
        continue
    if not assign_b:
        print(i)
        #qc_qbsc.rv(x_b[j_b], y_b[j_b], z_b[j_b], qubit=i)
        qc_qbsc.r(theta_b[j_b], phi_b[j_b], qubit=i)
        j_b+=1
        assign_b = True
        assign_a = False
        i+=4
        continue
        

In [ ]:
qc_qbsc.draw()

In [ ]:
# qc_qbsc.x(5)
# qc_qbsc.h(1)
# qc_qbsc.h(6)

In [ ]:
qc_qbsc.x(n_qubits_qbsc)
qc_qbsc.h(n_qubits_qbsc)

In [ ]:
# U3
j_b=0
for i in range(n_qubits_qbsc):
    if i % (j_b*5+1)==0 and i != 0:
        qc_qbsc.x(i)
        j_b+=1
#qc_qbsc.draw()

In [ ]:
# C1
i=0
while i < n_qubits_qbsc:
    #if i+5 > n_qubits_qbsc: break
    qc_qbsc.ccx(i, i+1, i+2)
    i = i+5
#qc_qbsc.draw()

In [ ]:
i=0
while i < n_qubits_qbsc:
    #if i+5 > n_qubits_qbsc: break
    qc_qbsc.x(i)
    qc_qbsc.x(i+1)
    i += 5
#qc_qbsc.draw()

In [ ]:
# C2
i=0
while i < n_qubits_qbsc:
    #if i+5 > n_qubits_qbsc: break
    qc_qbsc.ccx(i, i+1, i+3)
    i = i+5
#qc_qbsc.draw()

In [ ]:
i=0
while i < n_qubits_qbsc:
    qc_qbsc.x(i)
    i=i+5
#qc_qbsc.draw()

In [ ]:
qc_qbsc.barrier()
#qc_qbsc.draw()

In [ ]:
# C3 - check for domination in 1st, 2nd, etc, bits and use next one in row if a_i = b_i
i=2
while i < n_qubits_qbsc:
    qc_qbsc.x(i)
    qc_qbsc.x(i+1)
    i=i+4
i=2
while i < n_qubits_qbsc:
    qc_qbsc.ccx(i, i+1, i+2)
    i=i+4
i=2
while i < n_qubits_qbsc:
    qc_qbsc.x(i)
    qc_qbsc.x(i+1)
    i=i+4

In [ ]:
qc_qbsc.barrier()
qc_qbsc.draw()

In [ ]:
i = n_qubits_qbsc-1

while i - 5 > 0:
    print("here")
    qc_qbsc.ccx(i-1, i-4, i-6)
    qc_qbsc.ccx(i, i-4, i-5)
    qc_qbsc.barrier()
    i-=5
    print(i)

In [ ]:
qc_qbsc.cx(3, n_qubits_qbsc+1) # measure O2
qc_qbsc.x(2)
qc_qbsc.x(3)
qc_qbsc.ccx(2, 3, n_qubits_qbsc+1)
qc_qbsc.x(2)
qc_qbsc.x(3)
qc_qbsc.barrier()
qc_qbsc.cx(n_qubits_qbsc+1, n_qubits_qbsc)
qc_qbsc.barrier()
#qc_qbsc.measure(3, 2)
# qc_qbsc.measure(1, 0)
# qc_qbsc.measure(6, 1)

#mirror
qc_qbsc.x(2)
qc_qbsc.x(3)
qc_qbsc.ccx(2, 3, n_qubits_qbsc+1)
qc_qbsc.x(2)
qc_qbsc.x(3)
qc_qbsc.cx(3, n_qubits_qbsc+1) # measure O2
qc_qbsc.barrier()
qc_qbsc.draw()

In [ ]:
# if you measure O2
# simulator = Aer.get_backend('qasm_simulator')
# circ = transpile(qc_qbsc, simulator)

# # Run and get counts
# result = simulator.run(qc_qbsc, shots=1024).result()
# counts = result.get_counts(qc_qbsc)
# plot_histogram(counts)

In [ ]:
index_rand

In [ ]:
## MIRROR back
i = n_qubits_qbsc-1

while i - 5> 0:
    qc_qbsc.ccx(i, i-4, i-5)
    qc_qbsc.ccx(i-1, i-4, i-6)
    qc_qbsc.barrier()
    i-=5

In [ ]:
# C3 - check for domination in 1st, 2nd, etc, bits and use next one in row if a_i = b_i
i=2
while i < n_qubits_qbsc:
    qc_qbsc.x(i)
    qc_qbsc.x(i+1)
    i=i+4
i=2
while i < n_qubits_qbsc:
    qc_qbsc.ccx(i, i+1, i+2)
    i=i+4
i=2
while i < n_qubits_qbsc:
    qc_qbsc.x(i)
    qc_qbsc.x(i+1)
    i=i+4
qc_qbsc.barrier()

In [ ]:
i=0
while i < n_qubits_qbsc:
    qc_qbsc.x(i)
    i=i+5

In [ ]:
# C2
i=0
while i < n_qubits_qbsc:
    #if i+5 > n_qubits_qbsc: break
    qc_qbsc.ccx(i, i+1, i+3)
    i = i+5

In [ ]:
i=0
while i < n_qubits_qbsc:
    #if i+5 > n_qubits_qbsc: break
    qc_qbsc.x(i)
    qc_qbsc.x(i+1)
    i += 5

In [ ]:
# C1
i=0
while i < n_qubits_qbsc:
    #if i+5 > n_qubits_qbsc: break
    qc_qbsc.ccx(i, i+1, i+2)
    i = i+5

In [ ]:
# U3
j_b=0
for i in range(n_qubits_qbsc):
    if i % (j_b*5+1)==0 and i != 0:
        qc_qbsc.x(i)
        j_b+=1

In [ ]:
qc_qbsc.barrier()

In [ ]:
# qc_qbsc.measure(1, 0)
# qc_qbsc.measure(6, 1)

In [ ]:
simulator = Aer.get_backend('qasm_simulator')
circ = transpile(qc_qbsc, simulator)

# Run and get counts
result = simulator.run(qc_qbsc, shots=1024).result()
counts = result.get_counts(qc_qbsc)
plot_histogram(counts)

In [ ]:
index_rand

In [ ]:
## inversion around mean
qubit=1
controlled_qbits=[]
while qubit < n_qubits_qbsc:
    if qubit+5<n_qubits_qbsc:
        controlled_qbits.append(qubit)
    qc_qbsc.h(qubit) # apply H-gate
    qc_qbsc.x(qubit) # apply X-gate
    qubit+=5
print(qubit)
qc_qbsc.h(qubit-5)
qc_qbsc.mct(controlled_qbits, qubit-5)
qc_qbsc.h(qubit-5)

qubit=1
while qubit < n_qubits_qbsc:
    qc_qbsc.x(qubit) # apply H-gate
    qc_qbsc.h(qubit) # apply X-gate
    qubit+=5

In [ ]:
r_i = int(math.floor(np.pi/4*np.sqrt(n_qubits_qbsc)))
r_i

In [ ]:
# Grover

r_i = int(math.floor(np.pi/4*np.sqrt(2**n_qubits)))

qc_qbsc.measure(list([1, 6, 11]), list(range(n_qubits)))
qc_qbsc.draw()

In [ ]:
max_iters = int(math.ceil(np.sqrt(n_qubits_qbsc)))
max_iters

In [ ]:
r_i = int(math.floor(np.pi/4*np.sqrt(2**(n_qubits*2))))
r_i

In [ ]:
c=[]
for _ in range(1):

    # Transpile for simulator
    simulator = Aer.get_backend('qasm_simulator')
    circ = transpile(qc_qbsc, simulator)

    # Run and get counts
    result = simulator.run(qc_qbsc, shots=1024).result()
    counts = result.get_counts(qc_qbsc)
    c.append(counts)
    # #measure highest probability
    # probs = counts.items()
    # sorted_probs = dict(sorted(probs, key=lambda item: item[1], reverse=True))
    # sorted_probs_keys = list(sorted_probs.keys())
    # new_ix = [int(sorted_probs_keys[i],2) for i, _ in enumerate(sorted_probs_keys) if int(sorted_probs_keys[i],2) < k]
    # new_ix = new_ix[0]
    # threshold = distances[new_ix]

In [ ]:
plot_histogram(c[0])

In [ ]:
threshold

# YAAAAAAAAY!

In [ ]:
probs = counts.items()
probs
sorted_probs = dict(sorted(probs, key=lambda item: item[1], reverse=True))
sorted_probs_keys = list(sorted_probs.keys())
if sorted_probs_keys[0] == np.binary_repr(threshold, width=n_qubits):
    best = sorted_probs_keys[1]
else:
    best = sorted_probs_keys[0]
best
# new_threshold = sorted_probs_keys[0]

In [ ]:
sorted_probs_keys

In [ ]:
new_threshold

In [ ]:
plot_histogram(c[2])

In [ ]:
plot_histogram(c[3])

In [ ]:
thresh_bin, inputs

In [ ]:
index_rand

In [ ]:
distances[:20]

In [ ]:
distances = [[10,3,5,1], [4,2,6,9]]

## Test script for Grover

In [ ]:
# import scripts.QBSC_grover as qbscg
import importlib
importlib.reload(qbscg)

for d in distances[:10]:
    print(d)
    minimum, qc, qc_inv_mean, qc_total, counts = qbscg.grover_algo(d, int(latent_dim))
    #qc_total, counts = qbscg.grover_algo(d, int(latent_dim))
    #qc.draw()

In [ ]:
qc_total.draw()

In [ ]:
plot_histogram(counts)

In [ ]:
qc.draw()

In [ ]:
qc_inv_mean.draw()

In [ ]:
qc_total.draw()